<a target="_blank" href="https://colab.research.google.com/github/victorinho09/Trabajo_Fin_Grado/blob/main/TFGv2.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

First approach of a code base that loads a sample dataset and trains a basic neural network.

In [237]:
# EL problema estaba en el renombre del data_iris para conseguir las columnas,
#que las ponía a Nan, verificar que para el resto de datasets va bien
#Verificar numero de neuronas de salida para todos los datasets.

#Comprobar que lo ultimo metido de chatgpt no da error ya


!pip install ucimlrepo

#Imports
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import TensorBoard
from sklearn.impute import SimpleImputer

import datetime
from ucimlrepo import fetch_ucirepo

In [238]:
data_iris = fetch_ucirepo(id=53).data
data_heart_disease = fetch_ucirepo(id=45).data
data_adult = fetch_ucirepo(id=2).data
data_breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17).data
data_bank_marketing = fetch_ucirepo(id=222).data
data_diabetes = fetch_ucirepo(id=296).data

/usr/local/lib/python3.11/dist-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


In [239]:

def get_y_target_col(data_obj):
    """
    Dado data_obj = fetch_ucirepo(id=...).data, localiza la columna
    de 'targets' y la renombra a 'target'.
    Maneja varios casos:
     - Ya se llama 'target'
     - Se llama 'class'
     - Solo hay una columna en data.targets (la renombra a 'target')
    Devuelve un DataFrame con una sola columna llamada 'target'.
    """
    df_targets = data_obj.targets
    cols = list(df_targets.columns)

    # CASO 1: Si ya existe 'target'
    if 'target' in cols:
        y = df_targets[['target']].copy()
        print("Numero cols en y:", y.columns)
        return y

    # CASO 2: Si existe 'class'
    if 'class' in cols:
        y = df_targets[['class']].rename(columns={'class': 'target'})
        print("Numero cols en y:", y.columns)
        return y

    # CASO 3: Si solo hay 1 columna, la renombramos
    if len(cols) == 1:
        old_col = cols[0]
        y = df_targets.rename(columns={old_col: 'target'})
        return y[['target']]

def preprocessData(data):

    X = pd.DataFrame(data.features, columns=data.feature_names)
    y = get_y_target_col(data)

    # 2) Filtrar filas donde y['target'] sea NaN
    #    (esto crea una máscara True/False, y solo conservas las True)
    mask_not_nan = ~y['target'].isna()
    X = X[mask_not_nan].reset_index(drop=True)
    y = y[mask_not_nan].reset_index(drop=True)

    # 1) Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    #print(y_train.info())
    print("y_train count values:", y_train['target'].nunique())
    print("y_train values:", y_train['target'].unique())
    #print(y_train.head(30))

    '''
    print("X_train shape antes de imputer:", X_train.shape)
    print("¿NaNs en X_train?", X_train.isna().sum())
    '''

    # 4) One-Hot Encoding para y
    encoder = OneHotEncoder(sparse_output=False)
    y_train_encoded = encoder.fit_transform(y_train['target'].values.reshape(-1, 1))
    y_test_encoded  = encoder.transform(y_test['target'].values.reshape(-1, 1))
    #print("y_test_encoded: ", y_test_encoded[1])
    #print("size_y_test_encoded ", len(y_train_encoded[1]))

    # 5) Separar columnas numéricas y categóricas
    numeric_cols_train = X_train.select_dtypes(include=np.number).columns
    categorical_cols_train = X_train.select_dtypes(exclude=np.number).columns

    if len(numeric_cols_train) == 0 and len(categorical_cols_train) == 0:
      raise ValueError("No hay columnas numéricas ni categóricas en X_train")

    numeric_cols_test = X_test.select_dtypes(include=np.number).columns
    categorical_cols_test = X_test.select_dtypes(exclude=np.number).columns

    if len(numeric_cols_test) == 0 and len(categorical_cols_test) == 0:
      raise ValueError("No hay columnas numéricas ni categóricas en X_test")

    if(len(categorical_cols_train) == 0):
        print("NO hay columnas categoricas en X_train")

    # **Paso A**: Imputar numéricas
    imputer_num = SimpleImputer(strategy='mean')
    if len(numeric_cols_train) > 0:
      X_train_num_imputed = imputer_num.fit_transform(X_train[numeric_cols_train])
      X_test_num_imputed  = imputer_num.transform(X_test[numeric_cols_test])
    else:
      X_train_num = np.zeros((X_train.shape[0], 0))
      X_test_num  = np.zeros((X_test.shape[0], 0))

    # **Paso B**: Escalar numéricas
    scaler = StandardScaler()
    X_train_num_scaled = scaler.fit_transform(X_train_num_imputed)
    X_test_num_scaled  = scaler.transform(X_test_num_imputed)

    # **Paso C**: Imputar categóricas
    imputer_cat = SimpleImputer(strategy='most_frequent')
    if len(categorical_cols_train) > 0:
      X_train_cat_imputed = imputer_cat.fit_transform(X_train[categorical_cols_train])
      X_test_cat_imputed  = imputer_cat.transform(X_test[categorical_cols_test])
    else:
      X_train_cat_imputed = np.zeros((X_train.shape[0], 0))
      X_test_cat_imputed  = np.zeros((X_test.shape[0], 0))

    # **Paso D**: Codificar categóricas

    if(len(categorical_cols_train) != 0):
        print("Hay columnas categoricas en X_train")
        ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        X_train_cat_encoded = ohe.fit_transform(X_train_cat_imputed)
        X_test_cat_encoded  = ohe.transform(X_test_cat_imputed)

    else:
        X_train_cat_encoded = np.zeros((X_train.shape[0], 0))
        X_test_cat_encoded  = np.zeros((X_test.shape[0], 0))

    # **Paso E**: Concatenar numéricas escaladas + categóricas codificadas
    X_train_final = np.concatenate([X_train_num_scaled, X_train_cat_encoded], axis=1)
    X_test_final  = np.concatenate([X_test_num_scaled, X_test_cat_encoded],  axis=1)

    # Revisar si hay NaN en la salida final
    print("¿Hay NaN en X_train_final?", np.isnan(X_train_final).any())

    return X_train_final, X_test_final, y_train_encoded, y_test_encoded

#numero de neuronas rango (capa inicial) : raiz del numero de features - numero de features
#numero capas:




In [240]:
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
import math

# Creation of the ANN of example to try out the batch metrics in TensorBoard
def create_model(X_train,y_train,y_test):
  print("y_train_shape: ",y_train.shape[1])
  model = Sequential([
      Input(shape=(X_train.shape[1],)),
      Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Hidden Layer 1
      Dense(32, activation='relu'),  # Hidden Layer 2
      Dense(y_train.shape[1], activation='softmax')  # Output Layer with softmax for multiclass clasification
  ])

  # Compiling the model
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) #uso categorical_crossentropy cuando las etiquetas están codificadas con one-hot encoder. Si no usaría: sparse_categ_cross

  return model

class GlobalBatchLogger(Callback):
    def __init__(self, log_dir):
        super(GlobalBatchLogger, self).__init__()
        self.log_dir = log_dir
        self.global_step = 0

        # Variables para acumulados
        self.cumulative_loss = 0.0
        self.cumulative_accuracy = 0.0
        self.total_samples = 0  # para saber cuántas muestras se han procesado

    def on_train_begin(self, logs=None):
        # Creamos el escritor de resúmenes de TensorFlow al inicio del entrenamiento
        self.writer = tf.summary.create_file_writer(self.log_dir)
        self.writer.set_as_default()

    def on_batch_end(self, batch, logs=None):
        """
        Se llama al terminar cada batch.
        ‘batch’ es el índice local del batch en la época, pero usamos
        self.global_step para numerar de forma continua a lo largo
        de todo el entrenamiento.
        """
        if logs is not None:
            # Obtenemos las métricas *promediadas* que Keras ha calculado hasta este batch
            batch_loss = logs.get('loss', 0.0)
            batch_acc = logs.get('accuracy', 0.0)

            # Determinamos cuántas muestras se incluyeron en este batch
            batch_size = logs.get('size', None)
            if batch_size is None:
                # Si no aparece en logs, tomamos la configuración de la clase
                batch_size = self.params.get('batch_size', 1)

            # Actualizamos la suma acumulada de (loss * nº muestras) y (acc * nº muestras)
            self.cumulative_loss += batch_loss * batch_size
            self.cumulative_accuracy += batch_acc * batch_size
            self.total_samples += batch_size

            # Calculamos la media acumulada (hasta este batch)
            avg_cumulative_loss = self.cumulative_loss / self.total_samples
            avg_cumulative_accuracy = self.cumulative_accuracy / self.total_samples

            # Registramos exclusivamente la métrica acumulada
            tf.summary.scalar('cumulative_loss', data=avg_cumulative_loss, step=self.global_step)
            tf.summary.scalar('cumulative_accuracy', data=avg_cumulative_accuracy, step=self.global_step)

        # Forzamos la escritura en los ficheros de logs
        self.writer.flush()

        # Incrementamos el contador global de batches
        self.global_step += 1

    def on_train_end(self, logs=None):
        # Cerramos el writer al finalizar
        self.writer.close()

def compute_steps_for_batches(
    desired_batches,
    X_train_scaled,
    batch_size=16
):
    """
    Dado un número deseado de batches (desired_batches), calcula
    el número de steps que se pueden entrenar, sin pasarse
    del total de batches en X_train_scaled.
    - desired_batches: cuántos batches queremos.
    - X_train_scaled: datos de entrenamiento.
    - batch_size: tamaño de lote.

    Retorna steps_for_n_batches, que es el mínimo entre desired_batches y
    la cantidad real de batches que hay.
    """
    total_samples = X_train_scaled.shape[0]
    total_batches = math.ceil(total_samples / batch_size)

    # Para no pasarnos de la época, usamos el mínimo
    steps_for_n_batches = min(desired_batches, total_batches)

    # También puedes forzar que sea al menos 1
    if steps_for_n_batches < 1:
        steps_for_n_batches = 1

    return steps_for_n_batches

def train_and_evaluate(dataset,num_batches,log_dir,batch_size):
    X_train_scaled, X_test_scaled, y_train_encoded, y_test_encoded = preprocessData(dataset)
    model = create_model(X_train_scaled,y_train_encoded,y_test_encoded)

    tb_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
        update_freq='epoch'
    )
    global_batch_logger = GlobalBatchLogger(log_dir)


    steps_per_epoch = compute_steps_for_batches(num_batches,X_train_scaled,batch_size)

    history = model.fit(
        X_train_scaled,
        y_train_encoded,
        validation_data=(X_test_scaled, y_test_encoded),
        epochs=1,
        steps_per_epoch = steps_per_epoch,
        batch_size=batch_size,
        verbose=0,
        callbacks=[tb_callback,global_batch_logger]
    )
    loss, accuracy = model.evaluate(X_test_scaled, y_test_encoded)
    print(f"Steps_per_epoch: {steps_per_epoch},Pérdida: {loss:.4f}, Precisión: {accuracy:.4f}")
    return history

# **500 batches**

In [241]:
train_and_evaluate(data_iris,500, "logs/fit/iris/500batches",16)
train_and_evaluate(data_heart_disease,500, "logs/fit/heart_disease/500batches",16)

train_and_evaluate(data_breast_cancer_wisconsin_diagnostic,500, "logs/fit/breast_cancer_wisconsin_diagnostic/500batches",16)
train_and_evaluate(data_bank_marketing,500, "logs/fit/bank_marketing/500batches",16)
train_and_evaluate(data_diabetes,500, "logs/fit/diabetes/500batches",16)
train_and_evaluate(data_adult,500, "logs/fit/adult/500batches",16)


y_train count values: 4
y_train values: ['<=50K.' '>50K' '<=50K' '>50K.']
Hay columnas categoricas en X_train
¿Hay NaN en X_train_final? False
y_train_shape:  4


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


306/306 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5624 - loss: 0.9586
Steps_per_epoch: 500,Pérdida: 0.9590, Precisión: 0.5626


# **250 batches**

In [242]:

#Create the model for 250 batches
train_and_evaluate(
    dataset=data_iris,
    num_batches = 250,
    log_dir="logs/fit/iris/250batches",
    batch_size = 16
)
'''
train_and_evaluate(
    dataset=data_heart_disease,
    num_batches = 250,
    log_dir="logs/fit/heart_disease/250batches",
    batch_size = 16
)

train_and_evaluate(
    dataset=data_breast_cancer_wisconsin_diagnostic,
    num_batches = 250,
    log_dir="logs/fit/breast_cancer_wisconsin_diagnostic/250batches",
    batch_size = 16
)
train_and_evaluate(
    dataset=data_bank_marketing,
    num_batches = 250,
    log_dir="logs/fit/bank_marketing/250batches",
    batch_size = 16
)
train_and_evaluate(
    dataset=data_diabetes,
    num_batches = 250,
    log_dir="logs/fit/diabetes/250batches",
    batch_size = 16
)
train_and_evaluate(
    dataset=data_adult,
    num_batches = 250,
    log_dir="logs/fit/adult/250batches",
    batch_size = 16
)
'''

Numero cols en y: Index(['target'], dtype='object')
y_train count values: 3
y_train values: ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
NO hay columnas categoricas en X_train
¿Hay NaN en X_train_final? False
y_train_shape:  3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - accuracy: 0.6333 - loss: 0.9767
Steps_per_epoch: 8,Pérdida: 0.9767, Precisión: 0.6333


'\ntrain_and_evaluate(\n    dataset=data_heart_disease,\n    num_batches = 250,\n    log_dir="logs/fit/heart_disease/250batches",\n    batch_size = 16\n)\n\ntrain_and_evaluate(\n    dataset=data_breast_cancer_wisconsin_diagnostic,\n    num_batches = 250,\n    log_dir="logs/fit/breast_cancer_wisconsin_diagnostic/250batches",\n    batch_size = 16\n)\ntrain_and_evaluate(\n    dataset=data_bank_marketing,\n    num_batches = 250,\n    log_dir="logs/fit/bank_marketing/250batches",\n    batch_size = 16\n)\ntrain_and_evaluate(\n    dataset=data_diabetes,\n    num_batches = 250,\n    log_dir="logs/fit/diabetes/250batches",\n    batch_size = 16\n)\ntrain_and_evaluate(\n    dataset=data_adult,\n    num_batches = 250,\n    log_dir="logs/fit/adult/250batches",\n    batch_size = 16\n)\n'

# **100 batches**

In [243]:

#Create the model for 100 batches
train_and_evaluate(
    dataset=data_iris,
    num_batches = 100,
    log_dir="logs/fit/iris/100batches",
    batch_size = 16
)
'''
train_and_evaluate(
    dataset=data_heart_disease,
    num_batches = 100,
    log_dir="logs/fit/heart_disease/100batches",
    batch_size = 16
)

train_and_evaluate(
    dataset=data_breast_cancer_wisconsin_diagnostic,
    num_batches = 100,
    log_dir="logs/fit/breast_cancer_wisconsin_diagnostic/100batches",
    batch_size = 16
)
train_and_evaluate(
    dataset=data_bank_marketing,
    num_batches = 100,
    log_dir="logs/fit/bank_marketing/100batches",
    batch_size = 16
)
train_and_evaluate(
    dataset=data_diabetes,
    num_batches = 100,
    log_dir="logs/fit/diabetes/100batches",
    batch_size = 16
)
train_and_evaluate(
    dataset=data_adult,
    num_batches = 100,
    log_dir="logs/fit/adult/100batches",
    batch_size = 16
)

#problema en el numero de clases de salida (targets) de las neuronas en la perdida
'''

Numero cols en y: Index(['target'], dtype='object')
y_train count values: 3
y_train values: ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
NO hay columnas categoricas en X_train
¿Hay NaN en X_train_final? False
y_train_shape:  3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.8333 - loss: 0.9647
Steps_per_epoch: 8,Pérdida: 0.9647, Precisión: 0.8333


'\ntrain_and_evaluate(\n    dataset=data_heart_disease,\n    num_batches = 100,\n    log_dir="logs/fit/heart_disease/100batches",\n    batch_size = 16\n)\n\ntrain_and_evaluate(\n    dataset=data_breast_cancer_wisconsin_diagnostic,\n    num_batches = 100,\n    log_dir="logs/fit/breast_cancer_wisconsin_diagnostic/100batches",\n    batch_size = 16\n)\ntrain_and_evaluate(\n    dataset=data_bank_marketing,\n    num_batches = 100,\n    log_dir="logs/fit/bank_marketing/100batches",\n    batch_size = 16\n)\ntrain_and_evaluate(\n    dataset=data_diabetes,\n    num_batches = 100,\n    log_dir="logs/fit/diabetes/100batches",\n    batch_size = 16\n)\ntrain_and_evaluate(\n    dataset=data_adult,\n    num_batches = 100,\n    log_dir="logs/fit/adult/100batches",\n    batch_size = 16\n)\n\n#problema en el numero de clases de salida (targets) de las neuronas en la perdida\n'

In [244]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 4460), started 1:50:04 ago. (Use '!kill 4460' to kill it.)

<IPython.core.display.Javascript object>